# 1 Introduction

Structured Query Language, or **SQL**, is the programming language used with databases, and it is an important skill for any data scientist. In this course, you'll build your SQL using **BigQuery**, a web service that lets you apply SQL to huge datasets.

In this lesson, you'll learn the basics of accessing and examining BigQuery datasets. After you have a handle on these basics, we'll come back to build your SQL skills.

## 1.1 Your first BigQuery commands

To use BigQuery, we'll import the python package below:

In [1]:
from google.cloud import bigquery

The first step in the workflow is to create a `Client` object. As you'll soon see, this `Client` object will play a central role in retrieving information from BigQuery datasets.

In [2]:
# Create a "Client" object
client = bigquery.Client()

We'll work with a dataset of post on [Hacker News](https://news.ycombinator.com/), a website focusing on computer science and cybersecurity news.

In BigQuery, each dataset is contained in a corresponding project. In this case, our `hacker_news` dataset is contained in the `bigquery-public-data` project. To access the dataset,
* We begin by constructing a reference to the dataset with the `dataset()` method.
* Next, we use the `get_dataset()` method, along with the reference we just constructed, to fetch the dataset.

In [3]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Every dataset is just a collection of tables. You can think of a dataset as a spreadsheet file containing multiple tables, all composed of rows and columns.

We use the `list_tables()` method to list the tables in the dataset.

In [4]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:
    print(table.table_id)

full


Similar to how we fetched a dataset, we can fetch a table. in the code cell below, we fetch the `full` table in the `hacker_news` dataset.

In [5]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

In the next section, you'll explore the contents of this table in more detail. For now, take the time to use the image below to consolidate what you've learned so far.

![big-query](images\big-query.png)

## 1.2 Table schema

The structure of a table is called its `schema`. We need to understand a table's schema to effectively pull out the data we want.

In this example, we'll investigate the full table that we fetched above.

In [6]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', None, 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', None, 'Story url', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', None, 'Story or comment text', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', None, 'Is dead?', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', None, "The username of the item's author.", (), None),
 SchemaField('score', 'INTEGER', 'NULLABLE', None, 'Story score', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', None, 'Unix time', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', None, 'Timestamp for the unix time', (), None),
 SchemaField('type', 'STRING', 'NULLABLE', None, 'type of details (comment comment_ranking poll story job pollopt)', (), None),
 SchemaField('id', 'INTEGER', 'NULLABLE', None, "The item's unique id.", (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', None, 'Parent comment ID', (), None),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', N

Each `SchemaField` tells us about a specific column (which we also refer to as a **field**). In order, the information is:
* The **name** of the column
* The **field type** (or datatype) in the column
* The **mode** of the column (`'NULLABLE'` means that a column allows NULL values, and is the default)
* The **description** of the data in that column

The first field has the SchemaField:
`SchemaField('by', 'string', 'NULLABLE', "The username of the item's author.", ())`

This tell us:
* the field (or column) is called `by`,
* the data in this field is strings,
* NULL values are allowed, and
* it contains the usernames corresponding to each item's author.

We can use the `list_rows()` method to check just the first five lines of the `full` table to make sure this is right. (Sometimes databases have outdated descriptions, so it's good to check.) This returns a BigQuery `RowIterator` object that can quickly be converted to a pandas DataFrame with the `to_dataframe()` method.

In [7]:
import db_dtypes
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,"If the crocodile looked him up on Google, we b...",<NA>,raxxorrax,<NA>,1633421535,2021-10-05 08:12:15+00:00,comment,28756662,28750122,<NA>,<NA>,<NA>
1,None,None,What exactly are you looking for? I think Pyto...,<NA>,abiro,<NA>,1569141387,2019-09-22 08:36:27+00:00,comment,21040311,21040141,<NA>,<NA>,<NA>
2,None,None,"Ironically, this very project might help out w...",<NA>,mjevans,<NA>,1505769703,2017-09-18 21:21:43+00:00,comment,15279716,15276626,<NA>,<NA>,<NA>
3,None,None,As you start to gain some experience it can be...,<NA>,every_other,<NA>,1538575027,2018-10-03 13:57:07+00:00,comment,18130207,18128477,<NA>,<NA>,<NA>
4,None,None,"That’s what I was referring to, yes. I heard o...",<NA>,manmal,<NA>,1615664155,2021-03-13 19:35:55+00:00,comment,26449260,26449237,<NA>,<NA>,<NA>


The `list_rows()` method will also let us look at just the information in a specific column. If we want to see the first five entries in the `by` column, for example, we can do that!

In [8]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

,title
0,None
1,None
2,None
3,None
4,None


# 2 Select, From & Where

## 2.1 Introduction

Now that you know how to access and examine a dataset, you're ready to write your first SQL query! As you'll soon see, SQL queries will help you sort through a massive dataset, to retrieve only the information that you need.

We'll begin by using the keywords **SELECT**, **FROM**, and **WHERE** to get data from specific columns based on conditions you specify.

For clarity, we'll work with a small imaginary dataset `pet_records` which contains just one table, called `pets`.

![pets-table](images\pets-table.png)

## 2.2 SELECT ... FROM

The most basic SQL query selects a single column from a single table. To do this,
* specify the column you want after the word **SELECT**, and then
* specify the table after the word **FROM**.

For instance, to select the `Name` column (from the `pets` table in the `pet_records` database in the `bigquery-public-data` project), our query would appear as follows:

![select](images\select.png)

Note that when writing an SQL query, the argument we pass to **FROM** is *not* in single or double quotation marks (' or "). It is in backticks (`).

## 2.3 WHERE ...

BigQuery datasets are large, so you'll usuarlly want to return only the rows meeting specific conditions. You can do this using the **WHERE** clause.

The query below returns the entries from the `Name` column that are in rows where the `Animal` column has the text `'Cat'`.

![where](images\where.png)

## 2.4 Example: What are all the U.S. cities in the OpenAQ dataset?

Now that you've got the basics down, let's work through an example with a real dataset. We'll use an [OpenAQ](https://openaq.org/) dataset about air quality.

In [9]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the "openaq" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there's only one!)
for table in tables:  
    print(table.table_id)

global_air_quality


The dataset contains only one table, called `global_air_quality`. We'll fetch the table and take a peek at the first few rows to see what sort of data it contains.

In [10]:
# Construct a reference to the "global_air_quality" table
table_ref = dataset_ref.table("global_air_quality")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "global_air_quality" table
client.list_rows(table, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


Everything looks goog! So, let's put together a query. Say we want to select all the values from the `city` column that are in rows where the `country` column is `'US'` (for "United States").

In [11]:
# Query to select all the items from the "city" column where the "country" column is 'US'
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

Take the time now to ensure that this query lines up with what you learned above.

## 2.5 Submitting the query to the dataset

We're ready to use this query to get information from the OpenAQ dataset. As in the previous tutorial, the first step is to create a `Client` object.

In [12]:
# Create a "Client" object
client = bigquery.Client()

We begin by setting up the query with the query() method. We run the method with the default parameters, but this method also allows us to specify more complicated settings that you can read about in the documentation. We'll revisit this later.

In [13]:
# Set up the query
query_job = client.query(query)

Next, we run the query and convert the results to a pandas DataFrame

In [14]:
us_cities = query_job.to_dataframe()

In [17]:
# What five cities have the most measurements?
us_cities.city.value_counts().head()

city
Phoenix-Mesa-Scottsdale                     39414
Los Angeles-Long Beach-Santa Ana            27479
Riverside-San Bernardino-Ontario            26887
New York-Northern New Jersey-Long Island    25417
San Francisco-Oakland-Fremont               22710
Name: count, dtype: int64

## 2.6 More queries

If you want multiple columns, you can select them with a comma between the names:

In [18]:
query = """
        SELECT city, country
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

You can select all columns with a `*` like this:

In [19]:
query = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

## 2.7 Working with big datasets

BigQuery datasets can be huges. We allow you to do a lot of computation for free, but everyone has some limit.

Don't worry though: we'll teach you how to avoid scanning too much data at once, so that you don't run over your limit.

To begin, you can estimate the size of any query before running it. Here is an example using the (very large!) Hacker News dataset. To see how much data a query will scan, we create a `QueryJobConfig` object and set the `dry_run` parameter to `True`.

In [20]:
# Query to get the score column from every row where the type column has value "job"
query = """
            SELECT score, title
            FROM `bigquery-public-data.hacker_news.full`
            WHERE type = "jon"
        """

# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

This query will process 490833096 bytes.


You can also specify a parameter when running the query to limit how much data you are willing to scan. Here's an example with a low limit.

In [21]:
# Only run the query if it's less than 1 MB
ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)

# Set up the query (will only run if it's less than 1 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
safe_query_job.to_dataframe()

InternalServerError: 500 Query exceeded limit for bytes billed: 1000000. 491782144 or higher required.; reason: bytesBilledLimitExceeded, message: Query exceeded limit for bytes billed: 1000000. 491782144 or higher required.

Location: US
Job ID: bd40b979-e579-4ede-85dd-09e26dae08e3


In this case, the query was cancelled, because the limit of 1 MB was exceeded. However, we can increase the limit to run the query successfully!

In [22]:
# Only run the query if it's less than 1 GB
ONE_GB = 1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

# Set up the query (will only run if it's less than 1 GB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
job_post_scores = safe_query_job.to_dataframe()

# Print average score for job posts
job_post_scores.score.mean()

nan

# 3 Group By, Having & Count

Get more interesting insights directly from your SQL queries

## 3.1 Introduction

Now that you can select raw data, you're ready to learn how to group your data and count things within those groups. This can help you answer questions like:
* How many of each kind of fruit has our store sold?
* How many species of animal has the vet office treated?

To do this, you'll learn about three new tchniques: **GROUP BY**, **HAVING** and **COUNT()**. Once again, we'll use this made-up table of information on pets.

![groupby](images\groupby.png)

## 3.2 COUNT()

COUNT(), as you may have guessed from the name, returns a count of things. If you pass it the name of a column, it will return the number of entries in that column.

For instance, if we **SELECT** the **COUNT()** of the `ID` column in the `pets` table, it will return 4, because there are 4 ID's in the table.

![count](images\count.png)

**COUNT()** is an example of an **aggregate function**, which takes many values and returns one. (Other examples of aggregate functions include **SUM()**, **AVG()**, **MIN()**, and **MAX()**.) As you'll notice in the picture above, aggregate functions introduce strange columns names (lke `f0__`). Later in this tutorial, you'll learn how to change the name to something more descriptive.

## 3.3 GROUP BY
**GROUP BY** takes the name of one or more columns, and treats all rows with the same value in that column as a single group when you apply aggregate functions like **COUNT()**.

For example, say we want to know how many of each type of animal we have in the `pets` table. We can use GROUP BY to group together rows that have the same value in the `Animal` column, while using **COUNT()** to find out how many ID's we have in each group.

![count](images\groupby-1.png)

It returns a table with three rows (one for each distinct animal). We can see that the `pets` table contains 1 rabbit, 1 dog, and 2 cats.

## 3.4 Example: Which Hacker News comments generated the most discussion?

Ready to see an example on a real dataset? The Hacker News dataset contains information on stories and comments from the Hacker News social networking site.

In [23]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()


,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,"If the crocodile looked him up on Google, we b...",<NA>,raxxorrax,<NA>,1633421535,2021-10-05 08:12:15+00:00,comment,28756662,28750122,<NA>,<NA>,<NA>
1,None,None,What exactly are you looking for? I think Pyto...,<NA>,abiro,<NA>,1569141387,2019-09-22 08:36:27+00:00,comment,21040311,21040141,<NA>,<NA>,<NA>
2,None,None,"Ironically, this very project might help out w...",<NA>,mjevans,<NA>,1505769703,2017-09-18 21:21:43+00:00,comment,15279716,15276626,<NA>,<NA>,<NA>
3,None,None,As you start to gain some experience it can be...,<NA>,every_other,<NA>,1538575027,2018-10-03 13:57:07+00:00,comment,18130207,18128477,<NA>,<NA>,<NA>
4,None,None,"That’s what I was referring to, yes. I heard o...",<NA>,manmal,<NA>,1615664155,2021-03-13 19:35:55+00:00,comment,26449260,26449237,<NA>,<NA>,<NA>


Let's use the table to see which comments generate the most replies. Since:
* the `parent` column indicates the comment that was replied to, and
* the `id`column has the unique ID used to identify each comment,

We can **GROUP BY** the `parent` column and **COUNT()** the  `id` column in order to figure out the number of comments that were made as responses to a specific comment. (*This might not make sense immediately -- take your time here to ensure that everything is clear!*)

Furthermore, since we're only interested in popular comments, we'll look at comments with more than ten replies. So, we'll only returns groups **HAVING** more than ten ID's.

In [24]:
# Query to select comments that received more than 10 replies
query_popular = """
            SELECT parent, COUNT(id)
            FROM `bigquery-public-data.hacker_news.full`
            GROUP BY parent
            HAVING COUNT(id) > 10
            """

Now that our query is ready, let's run it and store the results in a pandas DataFrame:

In [26]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_popular, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
popular_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
popular_comments.head()

,parent,f0_
0,14505378,46
1,24895395,46
2,3878522,55
3,6176222,66
4,19208700,45


Each row in the `popular_comments` DataFrame corresponds to a comment that received more than ten replies. For instance, the comment with ID `801208` received `56` replies.

## 3.5 Aliasing and other improvements

A couple hints to make your queries even better:
* The column resulting from `COUNT(id)` was called `f0__`. That's not a very descriptive name. You can change the name by adding `AS NumPosts` after you specify the aggregation. This is called **aliasing**, and it will be covered in more detail in an upcoming lesson.
* If you are ever unsure what to put inside the **COUNT()** function, you can do `COUNT(1)` to count the rows in each group. Most people find it especially readable, because we know it's not focusing on other columns. It also scans less data than if supplied column names (making it faster and using less of your data access quota).

Using these tricks, we can rewrite our query:

In [27]:
# Improved version of earlier query, now with aliasing & improved readability
query_improved = """
                 SELECT parent, COUNT(1) AS NumPosts
                 FROM `bigquery-public-data.hacker_news.full`
                 GROUP BY parent
                 HAVING COUNT(1) > 10
                 """
                 
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_improved, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
improved_df = query_job.to_dataframe()

# Print the first five rows of the DataFrame
improved_df.head()

,parent,NumPosts
0,15009759,52
1,18608452,54
2,4941692,50
3,22018946,147
4,21941278,464


Now you have the data you want, and it has descriptive names. That's good style.

## 3.6 Note on using GROUP BY

Note that because it tells SQL how to apply aggregate functions (like **COUNT()**), it doesn't make sense to use **GROUP BY** without an aggregate function. Similarly, if you have any **GROUP BY** clause, then all variables must be passed to either a
1. **GROUP BY** command, or
2. an aggregation function.

Consider the query below:

In [28]:
query_good = """
             SELECT parent, COUNT(id)
             FROM `bigquery-public-data.hacker_news.full`
             GROUP BY parent
             """

Note that there are two variables: `parent` and `id`.
* `parent` was passed to a **GROUP BY** command (in `GROUP BY parent`), and
* `id` was passed to an aggregate function (in `COUNT(id)`).

And this query won't work, because the `author` column isn't passed to an aggregate function or a **GROUP BY** clause:

In [29]:
query_bad = """
            SELECT `by` AS author, parent, COUNT(id)
            FROM `bigquery-public-data.hacker_news.full`
            GROUP BY parent
            """

If make this error, you'll get the error message `SELECT list expression references column (column's name) which is neither grouped nor aggregated at`.

You may notice the ``by`` column in this query is surrounded by backticks. This is because **BY** is a reserved keyword used in clauses including **GROUP BY**. In BigQuery reserved keywords used as identifiers must be quoted in backticks to avoid an error. We also make subsequent references to this column more readable by adding an alias to rename it to `author`.